In [138]:
import plotly.graph_objects as go
import MetaTrader5 as mt
import numpy as np
import pandas as pd
import pandas_ta as ta
from datetime import datetime

In [139]:
mt.initialize()

True

In [140]:

df = pd.DataFrame(mt.copy_rates_range('EURUSD', 
                                            mt.TIMEFRAME_M5, 
                                            datetime(2023, 6, 1), 
                                            datetime.now()))

df['time'] = pd.to_datetime(df['time'], unit='s')

Test Multiple Indicators

In [141]:
## Functions to get indicators

## MA

def MA(df, n):

    '''n : rolling window'''

    ma = ta.ma('sma', df.close, length=n)
    return ma

## MACD

def MACD(df):

    exp1 = df.close.ewm(span=12).mean()
    exp2 = df.close.ewm(span=26).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=9).mean()

    return macd, signal

## Relative Strength Index

def RSI(df):

    rsi = ta.rsi(df.close, length=16)
    return rsi

## VWAP

def VWAP(df):

    vwap = ta.vwap(df.high, df.low, df.close, df.real_volume)
    return vwap

## EMA

def EMA(df):

    ema = ta.ema(df.close, length=100)
    return ema

## ATR

def ATR(df):
    atr = ta.atr(df.high, df.low, df.close, length=14)
    return atr

## BBANDS

def BBANDS(df):
    bbands = ta.bbands(df.close, length=14, stf=2.0)
    return bbands

In [142]:
## Apply indicators to dataframe

df['MA_21'] = MA(df, 21)
df['MA_50'] = MA(df, 50)
df['MA_100'] = MA(df, 100)

df['EMA'] = EMA(df)
df['ATR'] = ATR(df)
df['RSI'] = RSI(df)

In [151]:
## Get signals for every indicators

df['buy_ma_21'] = np.where((df.close.shift(1) < df.close) & (df.close.shift(1) < df.MA_21.shift(1)) & (df.close > df.MA_21), 1, 0)
df['sell_ma_21'] = np.where((df.close.shift(1) > df.close) & (df.close.shift(1) > df.MA_21.shift(1)) & (df.close < df.MA_21), -1, 0)

df['buy_ma_50'] = np.where((df.close.shift(1) < df.close) & (df.close.shift(1) < df.MA_50.shift(1)) & (df.close > df.MA_50), 1, 0)
df['sell_ma_50'] = np.where((df.close.shift(1) > df.close) & (df.close.shift(1) > df.MA_50.shift(1)) & (df.close < df.MA_50), -1, 0)

df['buy_ma_100'] = np.where((df.close.shift(1) < df.close) & (df.close.shift(1) < df.MA_100.shift(1)) & (df.close > df.MA_100), 1, 0)
df['sell_ma_100'] = np.where((df.close.shift(1) > df.close) & (df.close.shift(1) > df.MA_100.shift(1)) & (df.close < df.MA_100), -1, 0)

df['buy_ema'] = np.where((df.close.shift(1) < df.close) & (df.close.shift(1) < df.EMA.shift(1)) & (df.close > df.EMA), 1, 0)
df['sell_ema'] = np.where((df.close.shift(1) > df.close) & (df.close.shift(1) > df.EMA.shift(1)) & (df.close < df.EMA), -1, 0)

df['buy_rsi'] = np.where((df.RSI > 45) & (df.RSI.shift(1) < df.RSI), 1, 0)
df['sell_rsi'] = np.where((df.RSI < 55) & (df.RSI.shift(1) > df.RSI), -1, 0)


In [152]:
df

,time,open,high,low,close,tick_volume,spread,real_volume,MA_21,MA_50,...,buy_ma_21,sell_ma_21,buy_ma_50,sell_ma_50,buy_ma_100,sell_ma_100,buy_ema,sell_ema,buy_rsi,sell_rsi
0,2023-05-31 22:00:00,1.06739,1.06739,1.06703,1.06704,237,6,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2023-05-31 22:05:00,1.06704,1.06752,1.06703,1.06746,223,6,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2023-05-31 22:10:00,1.06746,1.06762,1.06723,1.06756,206,6,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2023-05-31 22:15:00,1.06756,1.06757,1.06733,1.06734,136,6,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2023-05-31 22:20:00,1.06734,1.06748,1.06719,1.06731,203,6,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,2023-06-02 23:35:00,1.07053,1.07055,1.07052,1.07053,28,6,0,1.070848,1.070999,...,0,0,0,0,0,0,0,0,0,0
594,2023-06-02 23:40:00,1.07053,1.07063,1.07053,1.07060,38,6,0,1.070819,1.070980,...,0,0,0,0,0,0,0,0,0,0
595,2023-06-02 23:45:00,1.07060,1.07065,1.07054,1.07058,71,6,0,1.070782,1.070965,...,0,0,0,0,0,0,0,0,0,-1
596,2023-06-02 23:50:00,1.07058,1.07060,1.07053,1.07058,61,6,0,1.070755,1.070950,...,0,0,0,0,0,0,0,0,0,0


In [153]:
## Moving Averages

fig = go.Figure(data=[
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    ),
    go.Scatter(
        x=df[df['buy_ma_21'] == 1].index,
        y=df[df['buy_ma_21'] == 1]['close'],
        mode='markers',
        marker=dict(size=10, color='Blue'),
        name='Signal_buy_ma_21'
    ),
    go.Scatter(
        x=df[df['sell_ma_21'] == -1].index,
        y=df[df['sell_ma_21'] == -1]['close'],
        mode='markers',
        marker=dict(size=10, color='Red'),
        name='Signal_sell_ma_21'
    ),
    go.Line(x=df.index, y=df['MA_21'], name='MA_21'
    ),
    go.Scatter(
        x=df[df['buy_ma_50'] == 1].index,
        y=df[df['buy_ma_50'] == 1]['close'],
        mode='markers',
        marker=dict(size=10, color='Green'),
        name='Signal_buy_ma_50'
    ),
    go.Scatter(
        x=df[df['sell_ma_50'] == -1].index,
        y=df[df['sell_ma_50'] == -1]['close'],
        mode='markers',
        marker=dict(size=10, color='Orange'),
        name='Signal_sell_ma_50'
    ),
    go.Line(x=df.index, y=df['MA_50'], name='MA_50'
    ),
    go.Scatter(
        x=df[df['buy_ma_100'] == 1].index,
        y=df[df['buy_ma_100'] == 1]['close'],
        mode='markers',
        marker=dict(size=10, color='Pink'),
        name='Signal_buy_ma_100'
    ),
    go.Scatter(
        x=df[df['sell_ma_100'] == -1].index,
        y=df[df['sell_ma_100'] == -1]['close'],
        mode='markers',
        marker=dict(size=10, color='Brown'),
        name='Signal_sell_ma_100'
    ),
    go.Line(x=df.index, y=df['MA_100'], name='MA_100')
])

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    paper_bgcolor='black',
    plot_bgcolor='black'
)
fig.update_xaxes(gridcolor='black')
fig.update_yaxes(gridcolor='black')

fig.show()


c:\Python311\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [154]:
## EMA

fig = go.Figure(data=[
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    ),
    go.Scatter(
        x=df[df['buy_ema'] == 1].index,
        y=df[df['buy_ema'] == 1]['close'],
        mode='markers',
        marker=dict(size=10, color='Green'),
        name='Signal_buy_ema'
    ),
    go.Scatter(
        x=df[df['sell_ema'] == -1].index,
        y=df[df['sell_ema'] == -1]['close'],
        mode='markers',
        marker=dict(size=10, color='Red'),
        name='Signal_sell_ema'
    ),
    go.Line(x=df.index, y=df['EMA'], name='EMA'
    )
])

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    paper_bgcolor='black',
    plot_bgcolor='black'
)
fig.update_xaxes(gridcolor='black')
fig.update_yaxes(gridcolor='black')

fig.show()

In [155]:
## RSI

fig = go.Figure(data=[
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    ),
    go.Scatter(
        x=df[df['buy_rsi'] == 1].index,
        y=df[df['buy_rsi'] == 1]['close'],
        mode='markers',
        marker=dict(size=10, color='Green'),
        name='Signal_buy_rsi'
    ),
    go.Scatter(
        x=df[df['sell_rsi'] == -1].index,
        y=df[df['sell_rsi'] == -1]['close'],
        mode='markers',
        marker=dict(size=10, color='Red'),
        name='Signal_sell_rsi'
    )
])

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    paper_bgcolor='black',
    plot_bgcolor='black'
)
fig.update_xaxes(gridcolor='black')
fig.update_yaxes(gridcolor='black')

fig.show()